### SCRAPING SOIL MONITORING

Faire petit résumé du notebook et de ce que fait le code

TO DO -> faire une version mac / linux et une version windows

In [ ]:
# Install required librairies
!pip install tldextract

You should consider upgrading via the '/Users/lucilerabeau/code/LucileRb/scraping-soil-monitoring/.venv/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
# Import librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
import time
from datetime import date
from datetime import datetime
import sys

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.proxy import *
from selenium.webdriver.common.proxy import Proxy, ProxyType

# processing
import tldextract
from urllib.parse import urlparse

In [ ]:
def scrapator(search_terms, nb_pages):
    
    """
    -----------------------------------------
    fonction pour scrapper serp google (SEO only)
    -----------------------------------------
    search_terms -> liste de search terms à utiliser
    nb_pages -> nombre de pages à scrapper (int)
    visible -> si on veut visualiser le browser ou non - par défaut = True donc visible (bool)
    (proxy -> proxy à utiliser (WIP))
    
    return -> df
    """

    start_time = datetime.now() # à formater
    print(f'Début du programme : {start_time}\n') # à formater

    df = pd.DataFrame()

    # initier webdriver
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options = options)

    print('***** scrapping en cours *****')

    # boucle sur search terms
    for term in search_terms:

        print(f'\n**** search term - {term} ****\n')

        # aller sur google
        driver.get('https://google.com')

        try:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#L2AGLb.tHlp8d'))).click()
        except:
            pass

        # petite pause
        time.sleep(round(random.uniform(1, 10), 1))

        # trouver la search bar
        elem = driver.find_element(By.NAME, 'q')
        elem.clear() # pour la vider si elle est pas vide

        # entrer element de recherche dans la search bar
        elem.send_keys(term)
        time.sleep(2)
        elem.send_keys(Keys.RETURN)

        # petite pause
        time.sleep(round(random.uniform(1, 10), 1))

        textes = []
        titres = []
        url = []
        image = []
        page_pos = []
        pages = []
        seaseo = []
        n = 0

        # prendre les infos des nb_pages pages
        while n < nb_pages:

            # compteur de page
            n += 1
            print(f'*** -- scrapping page {n} -- ***')

            # captcha
            #recaptcha(driver)
            time.sleep(100)

            # trouver tous les articles d'une page
            articles_SEO = driver.find_elements(By.CLASS_NAME, 'MjjYud') # -> SEO
            articles_SEA = driver.find_elements(By.CLASS_NAME, 'uEierd') # -> SEA
            position = 0

            # boucler sur articles SEO
            for article in articles_SEO:

                try:
                    # titre
                    title = article.find_element(By.TAG_NAME, 'h3')
                    titres.append(title.text)

                    # position
                    position += 1
                    page_pos.append(position)

                    # texte
                    textes.append(article.text)

                    # liens
                    link = article.find_element(By.TAG_NAME, 'a')
                    url.append(link.get_attribute('href'))

                    # page
                    pages.append(n)

                    # type de contenu
                    seaseo.append('SEO')

                    # Image
                    try:
                        img = article.find_element(By.CLASS_NAME, 'Z26q7c.UK95Uc.Sth6v')
                        image.append('Oui')
                    except:
                        image.append('Non')

                except:
                    titres.append('vide')
                    page_pos.append('vide')
                    textes.append('vide')
                    url.append('vide')
                    pages.append('vide')
                    image.append('vide')
                    seaseo.append('SEO vide')

            # boucler sur articles SEA
            position = 0
            for add in articles_SEA:

                # position
                position += 1
                page_pos.append(position)

                # texte
                textes.append(add.text)

                # titre
                try:
                    title = add.find_element(By.CLASS_NAME, 'CCgQ5.vCa9Yd.QfkTvb.N8QANc.MUxGbd.v0nnCb')
                    titres.append(title.text)
                except:
                    titres.append('erreur')

                # liens
                try:
                    link = add.find_element(By.TAG_NAME, 'a')
                    url.append(link.get_attribute('href'))
                except:
                    url.append('erreur')

                # page
                pages.append(n)

                # type de contenu
                seaseo.append('SEA')

                try:
                    img = add.find_element(By.CLASS_NAME, 'g-img.ZGomKf')
                    image.append('Oui')
                except:
                    image.append('Non')

            # se déplacer en bas de la page
            bottom_element = driver.find_element(By.ID, 'botstuff')
            bottom_element.location_once_scrolled_into_view

            # changer de page
            try:
                driver.find_element(By.XPATH, ("//*[contains(text(),'Suivant')]")).click()
            except:
                pass

            # petite pause
            time.sleep(round(random.uniform(1, 10), 1))

        # définir le bon indice à prendre quand pas le même nombre d'élements
        if len(titres) <= len(url) and len(titres) <= len(textes):
            size = len(titres)
        elif len(textes) <= len(url) and len(textes) <= len(titres):
            size = len(textes)
        elif len(url) <= len(textes) and len(url) <= len(titres):
            size = len(url)
        elif len(url) == len(textes) and len(url) == len(titres) and len(textes) == len(titres):
            print('même taille')
            size = len(url)
        else:
            print('problème quelque part')
            size = 0

        # mettre le tout dans un df
        df_term = pd.DataFrame()
        df_term['url'] = url[0:size]
        df_term['titles'] = titres[0:size]
        df_term['texte'] = textes[0:size]
        df_term['search_term'] = term
        df_term['pages'] = pages[0:size]
        df_term['page_position'] = page_pos[0:size]
        df_term['type'] = seaseo[0:size]
        df_term['images'] = image[0:size]

        df_term.drop(df_term.loc[df_term['titles'] == ''].index, inplace = True)
        df_term.reset_index(inplace = True, drop = True)

        df_term['company'] = df_term['url'].apply(lambda x : tldextract.extract(x).domain)
        df_term['domain'] = df_term['url'].apply(lambda x : urlparse(x).netloc)

        # Merger avec les données des messages précédents
        df = pd.concat([df, df_term], ignore_index = True)
        df['Date'] = date.today().strftime('%d/%m/%y')

        print(f'\nFin du scrapping pour le search term : {term} ')

    # quitter driver
    driver.close()

    end_time = datetime.now() # à formater
    print('Duration: {}'.format(end_time - start_time)) # à formater

    return df

#### RUN CODE
The execution of the following cell will run the principal code. Please select the query you wish to enter and the number of pages you want to scrap. Then, execute the cell.
Le programme va commencer et une nouvelle fenêtre de navigation internet va s'ouvrir. 

In [ ]:
mots = [
    'Monitor* AND Report* AND Verif* AND MRV AND soil*',
    #soil monitoring reporting verification'
    ]

df = scrapator(mots, 2) # test avec 50

Début du programme : 2025-02-12 22:14:17.854145

***** scrapping en cours *****

**** search term - Monitor* AND Report* AND Verif* AND MRV AND soil* ****

*** -- scrapping page 1 -- ***
*** -- scrapping page 2 -- ***

Fin du scrapping pour le search term : Monitor* AND Report* AND Verif* AND MRV AND soil* 
Duration: 0:03:33.359429
CPU times: user 84.9 ms, sys: 70.2 ms, total: 155 ms
Wall time: 3min 33s


In [ ]:
# Pour supprimer les lignes vides
df = df.loc[df['url'] != 'vide']
df = df.reset_index(drop = True)

In [44]:
df

,url,titles,texte,search_term,pages,page_position,type,images,company,domain,Date
0,https://scholar.google.fr/scholar?q=Monitor*+A...,Articles universitaires correspondant aux term...,Articles universitaires correspondant aux term...,Monitor* AND Report* AND Verif* AND MRV AND soil*,1,1,SEO,Non,google,scholar.google.fr,10/02/25
1,vide,vide,vide,Monitor* AND Report* AND Verif* AND MRV AND soil*,vide,vide,SEO vide,vide,vide,,10/02/25
2,https://www.horizon-europe.gouv.fr/monitoring-...,"Monitoring, reporting and verification of soil...","Monitoring, reporting and verification of soil...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,2,SEO,Non,horizon-europe,www.horizon-europe.gouv.fr,10/02/25
3,https://cordis.europa.eu/programme/id/HORIZON_...,"Monitoring, reporting and verification of soil...","Monitoring, reporting and verification of soil...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,3,SEO,Non,europa,cordis.europa.eu,10/02/25
4,https://www.deloitte.com/fr/fr/services/risk-a...,"How to quantify, verify, and manage the impact...","How to quantify, verify, and manage the impact...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,4,SEO,Non,deloitte,www.deloitte.com,10/02/25
...,...,...,...,...,...,...,...,...,...,...,...
412,https://www.morningstar.com/news/pr-newswire/2...,TACO BELL FEATURES NEARLY 400 REAL FANS IN ITS...,TACO BELL FEATURES NEARLY 400 REAL FANS IN ITS...,Monitor* AND Report* AND Verif* AND MRV AND soil*,50,3,SEO,Non,morningstar,www.morningstar.com,10/02/25
413,https://en.wikipedia.org/wiki/Greenhouse_gas_e...,Greenhouse gas emissions by Turkey,Greenhouse gas emissions by Turkey\nWikipedia\...,Monitor* AND Report* AND Verif* AND MRV AND soil*,50,4,SEO,Non,wikipedia,en.wikipedia.org,10/02/25
414,https://co2re.org/research-home/mrv-socioecolo...,"Monitoring, reporting and verification - CO₂RE","Monitoring, reporting and verification - CO₂RE...",Monitor* AND Report* AND Verif* AND MRV AND soil*,50,5,SEO,Non,co2re,co2re.org,10/02/25
415,vide,vide,vide,Monitor* AND Report* AND Verif* AND MRV AND soil*,vide,vide,SEO vide,vide,vide,,10/02/25


In [ ]:
# Liste des colonnes du tableau de données
df.columns

Index(['url', 'titles', 'texte', 'search_term', 'pages', 'page_position',
       'type', 'images', 'company', 'domain', 'Date'],
      dtype='object')

In [ ]:
# Les 20 domaines les plus présents dans les données
df['domain'].value_counts()[:20]

In [ ]:
# Export des résultats au format .csv
df.to_csv('../data/scraping_results_2025-02-10.csv', index = False)

In [ ]:
# Lire les résultats enregistrés
#df = pd.read_csv('../data/scraping_results_2025-02-10.csv')
#df

,url,titles,texte,search_term,pages,page_position,type,images,company,domain,Date
0,https://scholar.google.fr/scholar?q=Monitor*+A...,Articles universitaires correspondant aux term...,Articles universitaires correspondant aux term...,Monitor* AND Report* AND Verif* AND MRV AND soil*,1,1,SEO,Non,google,scholar.google.fr,10/02/25
1,https://www.horizon-europe.gouv.fr/monitoring-...,"Monitoring, reporting and verification of soil...","Monitoring, reporting and verification of soil...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,2,SEO,Non,horizon-europe,www.horizon-europe.gouv.fr,10/02/25
2,https://cordis.europa.eu/programme/id/HORIZON_...,"Monitoring, reporting and verification of soil...","Monitoring, reporting and verification of soil...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,3,SEO,Non,europa,cordis.europa.eu,10/02/25
3,https://www.deloitte.com/fr/fr/services/risk-a...,"How to quantify, verify, and manage the impact...","How to quantify, verify, and manage the impact...",Monitor* AND Report* AND Verif* AND MRV AND soil*,1,4,SEO,Non,deloitte,www.deloitte.com,10/02/25
4,https://www.sciencedirect.com/science/article/...,Solutions and insights for agricultural monito...,Solutions and insights for agricultural monito...,Monitor* AND Report* AND Verif* AND MRV AND soil*,1,6,SEO,Non,sciencedirect,www.sciencedirect.com,10/02/25
...,...,...,...,...,...,...,...,...,...,...,...
380,https://www.thenews.com.pk/print/1280468-ndc-l...,NDC lessons from the UAE and Brazil,NDC lessons from the UAE and Brazil\nThe News ...,Monitor* AND Report* AND Verif* AND MRV AND soil*,50,2,SEO,Non,thenews,www.thenews.com.pk,10/02/25
381,https://www.morningstar.com/news/pr-newswire/2...,TACO BELL FEATURES NEARLY 400 REAL FANS IN ITS...,TACO BELL FEATURES NEARLY 400 REAL FANS IN ITS...,Monitor* AND Report* AND Verif* AND MRV AND soil*,50,3,SEO,Non,morningstar,www.morningstar.com,10/02/25
382,https://en.wikipedia.org/wiki/Greenhouse_gas_e...,Greenhouse gas emissions by Turkey,Greenhouse gas emissions by Turkey\nWikipedia\...,Monitor* AND Report* AND Verif* AND MRV AND soil*,50,4,SEO,Non,wikipedia,en.wikipedia.org,10/02/25
383,https://co2re.org/research-home/mrv-socioecolo...,"Monitoring, reporting and verification - CO₂RE","Monitoring, reporting and verification - CO₂RE...",Monitor* AND Report* AND Verif* AND MRV AND soil*,50,5,SEO,Non,co2re,co2re.org,10/02/25


domain
en.wikipedia.org                 58
co2re.org                        30
www.newsngr.com.ng               29
www.morningstar.com              29
www.thenews.com.pk               29
www.youtube.com                   6
theoutcomesfund.com               3
trellis.net                       2
www.lse.ac.uk                     2
unfccc.int                        2
www.cesbio.cnrs.fr                2
www.proterrafoundation.org        2
climateactiontransparency.org     2
www.un-redd.org                   2
www.epa.gov                       2
untalent.org                      2
gggi.org                          2
cgspace.cgiar.org                 2
www.instagram.com                 2
www.carbon-drawdown.de            2
Name: count, dtype: int64